In [1]:
from pathlib import Path
import pydicom
import numpy as np
# import cv2
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

default_path = "C:\\Users\\write\\Desktop\\Medical_Images\\4_Projects\\Pneumonia_Classification\\rsna-pneumonia-detection-challenge\\"

In [2]:
labels = pd.read_csv(default_path + "stage_2_train_labels.csv")

labels["Target"].value_counts()

labels = labels.drop_duplicates("patientId")

labels["Target"].value_counts()

labels.columns

ROOT_PATH = Path(default_path + "stage_2_train_images/")
SAVE_PATH = Path(default_path + "Processed")


In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, models
from tqdm.notebook import tqdm
from PIL import Image
import os

# Set device (GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
device

device(type='cuda')

In [5]:
# Custom Dataset Class
class PneumoniaDataset(Dataset):
    def __init__(self, csv_file, image_dir, transform=None):
        self.labels = pd.read_csv(csv_file)
        self.image_dir = image_dir
        self.transform = transform
        
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        img_name = os.path.join(self.image_dir, f"{self.labels.iloc[idx, 0]}.dcm")  # Assuming image names match patient ID
        image = pydicom.dcmread(img_name).pixel_array
        image = Image.fromarray(image)  # Convert to PIL Image
        
        label = self.labels.iloc[idx, 1]  # Target label (0 or 1)
        
        if self.transform:
            image = self.transform(image)
        
        return image, label

In [6]:
# Image transformations for grayscale images
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.49], std=[0.25]),
    transforms.RandomAffine(degrees=(-5, 5),
                            translate=(0, 0.05),
                            scale=(0.9, 1.1)),
    transforms.RandomResizedCrop((224, 224), scale=(0.35, 1)),
    transforms.Grayscale(num_output_channels=3),  # Convert grayscale to 3 channels
])

In [7]:
# Dataset and DataLoader
train_dataset = PneumoniaDataset(csv_file=default_path + "stage_2_train_labels.csv", image_dir=ROOT_PATH, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Model: ResNet18
# Load the pre-trained ResNet18 model
import torch
import torch.nn as nn
from torchvision import models

# Initialize the ResNet18 model without pretrained weights
model = models.resnet18(pretrained=False)

# Change the first convolutional layer to accept 1 channel (grayscale images)
model.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)

# Modify the final fully connected layer for binary classification (1 output)
model.fc = nn.Linear(model.fc.in_features, 1)

# Move the model to the device (GPU or CPU)
model = model.to(device)

C:\Users\write\miniconda3\envs\pytorchenv\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\write\miniconda3\envs\pytorchenv\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [8]:
# Loss function and optimizer
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [ ]:
# Training loop
num_epochs = 35
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
        images, labels = images.to(device), labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs.squeeze(), labels.float())
        
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        # Compute accuracy
        preds = torch.round(torch.sigmoid(outputs))  # Since it's binary classification
        correct += (preds == labels).sum().item()
        total += labels.size(0)
    
    epoch_loss = running_loss / len(train_loader)
    epoch_acc = correct / total
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc*100:.2f}%")

Epoch 1/35:   0%|          | 0/945 [00:00<?, ?it/s]

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Save the entire model (architecture + weights)
torch.save(model, 'pneumonia_resnet18_full_model.pth')

In [ ]:
Loadedmodel = torch.load('pneumonia_resnet18_full_model.pth')
Loadedmodel = Loadedmodel.to(device)  # Send to GPU if needed

##  Loading the Model and Using It

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
model = model.